In [21]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool

In [22]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 3. Wrap it as a proper LangChain Tool
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,        # Use wikipedia.run, not wikipedia_retriever.run
    description="Lookup topics on Wikipedia; input is a search term."
)

In [23]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain.tools import Tool
arxiv_wrapper = ArxivAPIWrapper(
    top_k_results=3,
    load_max_docs=2,
    load_all_available_meta=False
)
arxiv_tool = Tool(
    name="arxiv",
    func=arxiv_wrapper.run,
    description=(
        "Search scientific papers on arXiv. Input should be a search query; "
        "returns summaries of the top results."
    ),
)

In [24]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [12]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "cpu"}
)


/tmp/ipykernel_13277/2877086701.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/home/adwait/anaconda3/envs/rag-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7325688f3d10>, search_kwargs={})

In [26]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description=(
        "Search for information about LangSmith. "
        "For any questions about LangSmith, you must use this tool!"
    )
)


In [74]:
tools = [wikipedia_tool, arxiv_tool]

In [75]:
##Agents - core idea is to use a llm to choose a sequence of steps
from langchain.agents import initialize_agent,AgentType
from langchain.llms.ollama import Ollama

llm = Ollama(model = "llama3.2")

In [76]:
# GOOD: Use initialize_agent directly
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    # Add other parameters like memory, return_intermediate_steps here
)



# BAD: Do not do this
# agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)


In [77]:
# Add this line before your invoke call to see the expected keys
print(agent.input_keys)


['input']


In [78]:
# Add this line to print the prompt template
print(agent.agent.llm_chain.prompt.template)


Answer the following questions as best you can. You have access to the following tools:

wikipedia(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[list[str]]' = None, metadata: 'Optional[dict[str, Any]]' = None, run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config: 'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any') -> 'Any' - Lookup topics on Wikipedia; input is a search term.
arxiv(query: str) -> str - Search scientific papers on arXiv. Input should be a search query; returns summaries of the top results.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [wikipedia, arxiv]
Action Input: the input to the action
Observation: the result of the action
... (thi

In [85]:
from langchain.agents.agent import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent_executor.invoke({"input": "Give me academic research articles about LangSmith from arXiv"
})




> Entering new AgentExecutor chain...
Thought: I need to look up information on "LangSmith" but I'm not sure what it refers to. Is it a person, a place, or something else?

Action: wikipedia
Action Input: LangSmith
Observation: Page: Will Smith
Summary: Willard Carroll  Smith II (born September 25, 1968) is an American actor, rapper, and film producer. Known for his work in both the screen and music industries, his accolades include an Academy Award, a Golden Globe Award, a BAFTA Award, and four Grammy Awards. Films in which he has appeared have grossed over $10 billion worldwide, making him one of Hollywood's most bankable stars.
Smith first gained recognition as part of a hip hop duo with DJ Jazzy Jeff, with whom he released five studio albums which contained five Billboard Hot 100-top 20 singles—"Parents Just Don't Understand", "A Nightmare on My Street", "Summertime", "Ring My Bell", and "Boom! Shake the Room"—from 1985 to 1994. He released the solo albums Big Willie Style (1997)

KeyboardInterrupt: 